# Web Scraping

We access data from the web all the time, but we always rely on the provider to get the data in a manner that is useful for us. This may not always be the case. Consider surfing a news page and having to look at many ads. Wouldn't it be nice to read the news without having to block the ads?

Or you might want to get your news in a conistent style but you have to rely on different news sites to get all your information.

Or you just need to copy some data from a website into another document or you have to do your own analysis of the data, but its only accessible in an HTML-table and you have to copy and paste every single entry from the table, this is quite cumbersome and also error prone.

Luckily computer scientists like to automate everything they can, with the additional ability to apply some logic to this process. Accessing data from a website with the aid of a programming language is called **web scraping**. Let's look at an example.

### Example

In the cell below you can find some code that extracts every item of a list in the given HTML string.

In [ ]:
from bs4 import BeautifulSoup

html = """
<ul>
    <li>Cat</li>
    <li>Dog</li>
    <li>Horse</li>
    <li>Cow</li>
</ul>    
"""

soup = BeautifulSoup(html, 'html.parser')

for item in soup.find_all('li'):
    print(item.string)

### Explanation

What going on here?

We take part of an HTML document and parse it in a `soup`. When we have the `soup` we have an object with which we can play. What this means is that we translated the HTML into something that python can understand, and now we can apply some logic to it.

This example might not be very exciting, but we will get to some better examples later on. Let's first understand the workflow we are dealing with here.

### Extra Bit: Writing to a File
Sometimes we may want to write the results of our webscraping to a text file. Again, this is something that python  can do naturally (The "w" means that our text overwrites any existing file. If we want to *append* something to the existing content, we replace the "w" with a "a".):

In [ ]:
with open("demofile2.txt", "w") as f:
    f.write("Now the file has content!")

In this case the file is written to the local binder that is created when you clicked on the link to open this notebook. If you caopy this code to a local python file and run it, the text file will be created in  the same folder as your python file.

We could now use this to write the previous results into an external file:

In [ ]:
from bs4 import BeautifulSoup

html = """
<ul>
    <li>Cat</li>
    <li>Dog</li>
    <li>Horse</li>
    <li>Cow</li>
</ul>    
"""

soup = BeautifulSoup(html, 'html.parser')

with open("animals.txt", "w") as f:
    for item in soup.find_all('li'):
        f.write(item.string + " ")

## Make it usable in Python

The basic idea here is to make the data available or usable in Python. Let's assume you want to develop a game and you need a random animal type. For now let's just work with this list, but we assume that there is a website with many animal types out there, where we can choose one randomly. Let's also assume that we have to do this task more than a million times, so it makes sense to automate this task.

In the cell above we already had access to the items of the list in Python, but it was not really a Python list, so we could not do all the thing we would like to. Let's change this is the next cell.

In [ ]:
from bs4 import BeautifulSoup

html = """
<ul>
    <li>Cat</li>
    <li>Dog</li>
    <li>Horse</li>
    <li>Cow</li>
</ul>    
"""

soup = BeautifulSoup(html, 'html.parser')

animals = [item.string for item in soup.find_all('li')]
print(animals)

## List Comprehensions

Most of the code here is the same as before. We don't have to do this, because we're runninig this in a Jupyter notebook, but it is good practice for now to try to understand everything we do in this cell.

After executing this cell, we have a list of all the animal types in Python. We achive this with the second to last line. This is called a list comprehension and is somewhat special to Python. However, it is quite easy to read, once you understand the syntax.

We tell Python that we want a list because we use the `[]`-brackets. Inside the bracktes we have to look at the part from `for` until the end of the bracket. You can see that this is exactly the same as in the cell above, and you already know that this means we loop over each `<li>` and assign it the name `item`. When you look at the beginning of the bracket, you see that we use `item.string` which is the thing that we printed in the cell before. Now we tell Python that this is what should make up the list.

List comprehensions are used quite often in Python, so you should get familiar with them.

## Choosing a Random Element

We can easily choose a random element from a list in Python with the following code.

In [ ]:
from random import choice

print(choice(animals))

The `choice` function from Python chooses one random element from a list. When we also use the `print` function, we can output the element to the commandline.

## Loading from a URL

In a real world scenario we would not enter the HTML into our code directly, we would want to read the HTML from a url. We can easily do this with Python with the help of the `requests` library.

In the next cell you can find an example where we access a wikipedia article to get a list of bears.

In [ ]:
import requests

url = "https://en.wikipedia.org/wiki/List_of_ursids"

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

bears = [elem.string for elem in soup.select('th[scope=row] a')]
print(bears)

## Bad Result

This result does not look as nice as before. This is a general problem when extracting information from a website. Usually the website is made to read the data by the visitor from the rendered HTML document and not with a script from the pure HTML.

Let's clean up the list in the next cell.

In [ ]:
def filter_func(elem):
    if elem == None:
        return False
    if ' ' in elem:
        return True
    return False

bears = list(filter(filter_func, bears))
bears

## Filter Function

We can use the `filter` function from Python to clean out the list. We pass a function to the filter that decides if an element should be in the list or not, then we can implement our logic for cleaning the data. 

The logic here is very simple. If the item is a `None` we exclude it from the list. The other values are a bit strange, but if we look quickly at the list, we see that all bear names have at least one space in it, so we only return True when a space occurs in the name. We could also do this with the string `bear`, but then we would lose the 'Giant Panda'.

Filtering in this manner is a bit messy, but we don't care about this right now, because we just want to get the data quickly and use whatever tools we have at our disposable.

## Selecting Elements

To select elements from an HTML filie we can use the `select` function that `BeautifulSoup` provides for us. We can pass the `select` function any CSS selector we know. Up to now you probably used some simple CSS selectors with just tag names or classes, but now it makes sense to build more complex CSS selctors. To see what is available, just read the documentation [here](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Selectors).

## A more practical Example

So the example we followed so far was a bit strange. Let's take a real world example that we can only solve with web scraping.

Let's consider the following situation. You have two new cats and you are very bad at coming up with some names. The *only* solution is to use a random name generator that we feed with cat names from a website we found. Look at the following [website](https://www.rover.com/blog/cat-names/#full-list) with some cat names. We want to read in the list of all these names, and then randomly show some cat names until we have found what we are looking for.

In [ ]:
from bs4 import BeautifulSoup
import requests
from random import choice

url = "https://www.rover.com/blog/cat-names/#full-list"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

cats = [item.string for item in soup.select(".name-list-wrapper li")]
print(f"Found a total of {len(cats)} cat names.")

## Build the Selector

How to build this selector is not trivial. This depends on the website how things are structured there. For this website and this use case, it works quite good. There are some duplicates, but we don't care that much for now.

To find a possible selector, we visit the website and right-click an element that we are interested is. We then look at the DOM to see what properties this element has, and how we can address is with a selector.

## Choosing a Name randomly

To finish this very important application we now have to select a name at random from this list. We can do this again with the `choice` function from Python.

In [ ]:
print(choice(cats))

## Filtering the List

We can also apply filters for some logic, and not just to clean up the data. Let's say we only want very very long names. We can just write a filter function that only selects items with more than 8 characters.

In [ ]:
def f_func(item):
    if len(item) > 8:
        return True
    return False

new_cats = list(filter(f_func, cats))
print(new_cats)

As you can see, there are not that many cat names on this website with more than 8 characters. Maybe we should apply some other rules. Let's say we only want names that start with a 'B'. Then we could apply the following filter from the next cell.

In [ ]:
def f_func(item):
    if item[0] == 'B':
        return True
    return False

new_cats = list(filter(f_func, cats))
print(new_cats)

### Exercise

Develop your own filter to play around with this list. Possible ideas are:

- Names that start with a certain character.
- Names that end with a certain character.
- Names that are exactly 5 characters long.
- Names that start and end with the same character.
- Names that don't have the letter 'i' in them.
- Names that are an anagram.

### Exercise

Search or another website from which you would like to pull some, data, then develop your own webscraping idea.

## A very useful Tool

The examples so far are not very convincing. Maybe this is something more useful for you. The following 2 cells implement a vocabulary learning game. Try it out and configure it to your needs.

In [ ]:
from bs4 import BeautifulSoup
import requests
from random import choice

url = "https://www.sprachheld.de/grundwortschatz-franzoesisch-wichtigste-vokabeln/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

dic = [item 
       for item in soup.select("tr")]
print(f"Found a total of {len(dic)} vocabulary entries.")

In [ ]:
elem = choice(dic)
c = elem.select("td")
inp = input(f"Translate '{c[0].string}' to french: ")
ans = c[1].string
if inp.lower() == ans.lower():
    print(f"Correct, the answer is {ans}.")
else:
    print(f"This is WRONG!!! The correct answer would be: {ans}.")